# Imports

In [33]:
import numpy as np
import cv2
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# Carregando o banco de dados de imagens com Mascara e sem Mascara na variavel

In [34]:
with_mask = np.load('with_mask.npy')
without_mask = np.load('without_mask.npy')

with_mask = with_mask.reshape(200, 50 * 50 * 3)
without_mask = without_mask.reshape(200, 50 * 50 * 3)

# Concatenar número

In [35]:
X = np.r_[with_mask, without_mask]

# Definindo as Labels

In [36]:
labels = np.zeros(X.shape[0])
labels[200:] = 1.0

# Identificando os tipos de opções

In [37]:
names = {0 : 'C/ mascara', 1 : 'Sem mascara'}

# Componente de analise

In [38]:
pca = PCA(n_components = 3)
x_train = pca.fit_transform(x_train)

# Treinamento da maquina

In [39]:
x_train, x_test, y_train, y_test = train_test_split(X, labels, test_size=0.3)

# Definição do SVC

In [40]:
svm = SVC()
svm.fit(x_train, y_train)

SVC()

# Taxa de acerto

In [41]:
y_pred = svm.predict(x_test)
accuracy_score(y_test, y_pred)

0.9833333333333333

# Detector de mascara

In [43]:
haar_data = cv2.CascadeClassifier('data.xml')
capture = cv2.VideoCapture(0)
data = []
font = cv2.FONT_HERSHEY_COMPLEX
while True:
    flag, img = capture.read()
    faces = haar_data.detectMultiScale(img)
    for x,y,w,h in faces:
        cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,255), 4)
        face = img[y:y+h, x:x+w, :]
        face = cv2.resize(face, (50,50))
        face = face.reshape(1,-1)
        # face = pca.transform(face)
        pred = svm.predict(face)
        n = names[int(pred)]
        cv2.putText(img, n, (x,y), font, 1, (244,250,250), 2)
        # print(n)
    cv2.imshow('Automatiza de A a Z', img)
    if cv2.waitKey(2) == 27:
        break
capture.release()
cv2.destroyAllWindows()